In [ ]:
from resources.workspace import *

# The ensemble (Monte-Carlo) approach
is an approximate method for doing Bayesian inference. Instead of computing the full posterior distributions, we instead try to generate ensembles from them.

An ensemble is an *iid* sample. I.e. a set of "members" ("particles", "realizations", or "sample points") that have been drawn ("sampled") independently from the same distribution. With the EnKF, these assumptions are generally tenous, but pragmatic.

Ensembles can be used to characterize uncertainty: either by reconstructing (estimating) the distribution from which it is assumed drawn, or by computing various *statistics* such as the mean, median, variance, covariance, skewness, confidence intervals, etc (any function of the ensemble can be seen as a "statistic"). This is illustrated by the code below.

In [ ]:
# Parameters
b   = 0
B   = 25    
B12 = sqrt(B)

def true_pdf(x):
    return ss.norm.pdf(x,b,sqrt(B))

# Plot true pdf
xx = 3*linspace(-B12,B12,201)
fig, ax = plt.subplots()
ax.plot(xx,true_pdf(xx),label="True");

# Sample and plot ensemble
M = 1   # length of state vector
N = 100 # ensemble size
E = b + B12*randn((N,M))
ax.plot(E, zeros(N), '|k', alpha=0.3, ms=100)

# Plot histogram
nbins = max(10,N//30)
heights, bins, _ = ax.hist(E,normed=1,bins=nbins,label="Histogram estimate")

# Plot parametric estimate
x_bar = np.mean(E)
B_bar = np.var(E)
ax.plot(xx,ss.norm.pdf(xx,x_bar,sqrt(B_bar)),label="Parametric estimate")

ax.legend();

# Uncomment AFTER Exc 4:
# dx = bins[1]-bins[0]
# c = 0.5/sqrt(2*pi*B)
# for height, x in zip(heights,bins):
#     ax.add_patch(mpl.patches.Rectangle((x,0),dx,c*height/true_pdf(x+dx/2),alpha=0.3))
# Also set
#  * N = 10**4
#  * nbins = 50

**Exc 2:** Which approximation to the true pdf looks better: Histogram or the parametric?   
Does one approximation actually start with more information? The EnKF takes advantage of this.

**Exc 4*:** Suppose the histogram bars get normalized (divided) by the value of the pdf at their location.  
How do you expect the resulting histogram to look?  
Test your answer by uncommenting the block in the above code.

#### Exc 5*:
Use the method of `gaussian_kde` from `scipy.stats` to make a "continuous histogram" and plot it above.
`gaussian_kde`  

In [ ]:
#show_answer("KDE")

**Exc 6 (Multivariate Gaussian sampling):**
Suppose $\mathbf{z}$ is a standard Gaussian,
i.e. $p(\mathbf{z}) = \mathcal{N}(\mathbf{z} \mid 0,\mathbf{I}_M)$,
where $\mathbf{I}_M$ is the $M$-dimensional identity matrix.  
Let $\mathbf{x} = \mathbf{L}\mathbf{z} + \mathbf{b}$. 
Recall [Exc 3.1](T3%20-%20Univariate%20Kalman%20filtering.ipynb#Exc-3.1:),
which yields $p(\mathbf{x}) = \mathcal{N}(\mathbf{x} \mid \mathbf{b}, \mathbf{L}^{}\mathbf{L}^T)$.
    
 * (a). $\mathbf{z}$ can be sampled using `randn((M,1))`. How (where) is `randn` defined?
 * (b). Consider the above definition of $\mathbf{x}$ and the code below.
 Complete it so as to generate a random realization of $\mathbf{x}$.  
 Hint: matrix-vector multiplication can be done using the symbol `@`. 

In [ ]:
M   = 3 # ndim
b   = 10*ones(M)
B   = diag(1+arange(M))
L   = np.linalg.cholesky(B) # B12
print("True mean and cov:")
print(b)
print(B)

### INSERT ANSWER (b) ###

In [ ]:
#show_answer('Gaussian sampling a')

In [ ]:
#show_answer('Gaussian sampling b')

 * (c). Now sample $N = 100$ realizations of $\mathbf{x}$
 and collect them in an $M$-by-$N$ "ensemble matrix" $\mathbf{E}$.  
 Avoid `for` loops (the main thing to figure out is:
 how to add a (mean) vector to a matrix).

In [ ]:
N  = 100 # ensemble size

E = ### INSERT ANSWER (c) ###

# Use the code below to assess whether you got it right
x_bar = np.mean(E,axis=1)
B_bar = np.cov(E)
print("Estimated mean and cov:")
with printoptions(precision=1):
    print(x_bar)
    print(B_bar)
plt.matshow(B_bar,cmap="Blues"); plt.grid('off'); plt.colorbar()

In [ ]:
#show_answer('Gaussian sampling c')

**Exc 8*:** How erroneous are the ensemble estimates on average?

In [ ]:
#show_answer('Average sampling error')

**Exc 10:** Given the previous ensemble matrix $\mathbf{E}$, compute its sample mean $\overline{\mathbf{x}}$ and covariance matrix, $\overline{\mathbf{B}}$:
$$ \overline{\mathbf{x}} = \frac{1}{N}   \sum_{n=1}^N \mathbf{x}_n \\
   \overline{\mathbf{B}} = \frac{1}{N-1} \sum_{n=1}^N (\mathbf{x}_n - \overline{\mathbf{x}}) (\mathbf{x}_n - \overline{\mathbf{x}})^T  $$

In [ ]:
# Don't use numpy's mean, cov
def estimate_mean_and_cov(E):
    M, N = E.shape
    
    ### INSERT ANSWER ###
    
    return x_bar, B_bar

x_bar, B_bar = estimate_mean_and_cov(E)
print(x_bar)
print(B_bar)

In [ ]:
#show_answer('ensemble moments')

**Exc 12:** Why is the normalization by $(N-1)$ for the covariance computation?

In [ ]:
#show_answer('Why (N-1)')

**Exc 14:** Like Matlab, Python (numpy) is quicker if you "vectorize" loops. This is emminently possible with computations of ensemble moments. 
 * (a). Let $\mathbf{X} = \begin{bmatrix}
		\mathbf{x}_1 -\mathbf{\bar{x}}, & \ldots & \mathbf{x}_n -\mathbf{\bar{x}}, & \ldots & \mathbf{x}_N -\mathbf{\bar{x}}
	\end{bmatrix} \, .
	$
Show that $\overline{\mathbf{B}} = \mathbf{X} \mathbf{X}^T /(N-1)$.
 * (b). Code up this formula for $\overline{\mathbf{B}}$ and insert it in `estimate_mean_and_cov(E)`

In [ ]:
#show_answer('ensemble moments vectorized')

**Exc 16:** Implement the cross-covariance estimator $\overline{Cov(\mathbf{x},\mathbf{y})} = \frac{1}{N-1} \sum_{n=1}^N (\mathbf{x}_n - \overline{\mathbf{x}}) (\mathbf{y}_n - \overline{\mathbf{y}})^T $.  
If you can, use a vectorized form similarly to Exc 14a. 

In [ ]:
def estimate_cross_cov(Ex,Ey):
    ### INSERT ANSWER ###

In [ ]:
#show_answer('estimate cross')

**Exc 18 (error notions)*:**
 * (a). What's the difference between error residual?
 * (b). What's the difference between error and bias?
 * (c). Show `MSE = RMSE^2 = Bias^2 + Var`

In [ ]:
#show_answer('errors')

### Next: [Writing your own EnKF](T8 - Writing your own EnKF.ipynb)